## The Original EigenDecompression.EigenDecompose

See

* https://mobile.twitter.com/realize_ss/status/1615160291108745216
* https://qiita.com/lelele/items/8408410a94f5c6b8f76e

In [1]:
using LinearAlgebra
M = rand(100,100)#対角化したい行列
E, P = eigen(M)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
100-element Vector{ComplexF64}:
  -2.754451450502443 - 0.39749988627169797im
  -2.754451450502443 + 0.39749988627169797im
 -2.5250571636150148 - 0.6374236149840182im
 -2.5250571636150148 + 0.6374236149840182im
  -2.356868242449035 + 0.0im
 -2.2156064058331095 - 1.4230980003258915im
 -2.2156064058331095 + 1.4230980003258915im
 -2.2010456461956913 - 0.2927949755547112im
 -2.2010456461956913 + 0.2927949755547112im
  -2.183471618894222 - 0.7964116773484528im
  -2.183471618894222 + 0.7964116773484528im
 -1.9439605762180874 - 1.7634788581232392im
 -1.9439605762180874 + 1.7634788581232392im
                     ⋮
  1.6711235830269149 - 1.1869067408760747im
  1.6711235830269149 + 1.1869067408760747im
   1.801639489578879 - 0.15506751202355196im
   1.801639489578879 + 0.15506751202355196im
   2.191927392912879 - 1.206248996604773im
   2.191927392912879 + 1.206248996604773im
  2.2857563767873113 + 0.0im
  2.52706455837

In [2]:
exp(eigen(M))

LoadError: MethodError: no method matching exp(::Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}})

[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m)
[0m[90m   @[39m [90mBase[39m [90mspecial\[39m[90m[4mexp.jl:325[24m[39m
[0m  exp([91m::Adjoint{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:595[24m[39m
[0m  exp([91m::Transpose{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:596[24m[39m
[0m  ...


In [3]:
module EigenDecompression

export EigenDecompose, eigDecomp
using LinearAlgebra
import Base.*, Base./

#対角化された行列型
struct EigenDecompose{T<:Number} <: AbstractMatrix{T}
    P::AbstractMatrix{T}
    D::Diagonal{T}
    invP::AbstractMatrix{T}
end

#普通のMatrixを対角化する
function eigDecomp(mat::AbstractMatrix)
    E, P = eigen(mat)
    EigenDecompose(P, Diagonal(E), inv(P))
end

#EigenDecompose型に対する関数
Base.exp(eig::EigenDecompose) = EigenDecompose(eig.P, exp(eig.D), eig.invP)
*(eig::EigenDecompose, vec::AbstractVector) = eig.P * eig.D * eig.invP * vec
*(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D*sc, eig.invP)
/(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D/sc, eig.invP)

#普通のMatrixに戻す
Base.Array(eig::EigenDecompose) = eig.P * eig.D * eig.invP

end

Main.EigenDecompression

In [4]:
using .EigenDecompression

M = rand(100, 100)
eM = eigDecomp(M)
for i in 1:100
    v = rand(100)
    rnd = rand()
    @assert exp(M*rnd)*v ≈ exp(eM*rnd)*v
end

In [5]:
using BenchmarkTools

M = rand(100, 100);

#普通な方
function bench1(M)
    for i in 1:100
        v = rand(100)
        exp(M*rand())*v
    end
end

#今回実装した方
function bench2(M)
    eM = eigDecomp(M)
    for i in 1:100
        v = rand(100)
        exp(eM*rand())*v
    end
end

bench2 (generic function with 1 method)

In [6]:
@benchmark bench1(M)

BenchmarkTools.Trial: 61 samples with 1 evaluation.
 Range (min … max):  74.035 ms … 96.327 ms  ┊ GC (min … max): 2.37% … 4.74%
 Time  (median):     82.196 ms              ┊ GC (median):    2.31%
 Time  (mean ± σ):   83.068 ms ±  5.198 ms  ┊ GC (mean ± σ):  3.43% ± 1.96%

          ▃ ▃ ▃ █   █▃▃█  ▃▃ █  ▃   ▃     ▃      ▃   ▃         
  ▇▇▁▇▁▁▇▇█▁█▇█▇█▁▁▇████▁▇██▇█▁▇█▁▇▁█▇▇▁▇▇█▁▁▁▇▇▇█▇▇▁█▁▁▇▁▇▁▇ ▁
  74 ms           Histogram: frequency by time        93.2 ms <

 Memory estimate: 53.80 MiB, allocs estimate: 1900.

In [7]:
@benchmark bench2(M)

BenchmarkTools.Trial: 707 samples with 1 evaluation.
 Range (min … max):  6.119 ms …  11.841 ms  ┊ GC (min … max): 0.00% … 29.23%
 Time  (median):     6.887 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.062 ms ± 711.919 μs  ┊ GC (mean ± σ):  0.44% ±  2.03%

     ▄▇▆█▅▅▅▄▁▂  ▁▁▅▄                                          
  ▂▆▆████████████████▇███▇▄▆▅▅▃▅▄▃▃▂▃▂▃▃▃▃▂▃▂▂▃▂▃▂▁▃▁▁▁▃▁▂▁▂▂ ▄
  6.12 ms         Histogram: frequency by time         9.8 ms <

 Memory estimate: 1.82 MiB, allocs estimate: 1728.

## EigenDecomposedMatrices.EigenDecomposed

In [8]:
using LinearAlgebra
using BenchmarkTools

In [9]:
module EigenDecomposedMatrices

export EigenDecomposed

using LinearAlgebra
using Memoization

struct EigenDecomposed{
        T,
        TE<:AbstractVector{T},
        TP<:AbstractMatrix{T},
        TinvP<:AbstractMatrix{T}
    } <: AbstractMatrix{T}
    E::TE
    P::TP
    invP::TinvP
end

function EigenDecomposed(A::AbstractMatrix)
    E, P = eigen(A)
    invP = ishermitian(A) ? P' : inv(P)
    EigenDecomposed(E, P, invP)
end

LinearAlgebra.eigvals(ed::EigenDecomposed) = ed.E
LinearAlgebra.eigvecs(ed::EigenDecomposed) = ed.P
inveigvecs(ed::EigenDecomposed) = ed.invP
@memoize Base.parent(ed::EigenDecomposed) = eigvecs(ed) * Diagonal(eigvals(ed)) * inveigvecs(ed)
Base.convert(::Type{Array}, ed::EigenDecomposed) = convert(Array, parent(ed))
for op in (:eltype, :size)
    @eval Base.$op(ed::EigenDecomposed) = $op(eigvecs(ed))
end
Base.getindex(ed::EigenDecomposed, I...) = getindex(parent(ed), I...)

Base.:*(c::Number, ed::EigenDecomposed) = EigenDecomposed(c*eigvals(ed), eigvecs(ed), inveigvecs(ed))
Base.:*(ed::EigenDecomposed, c::Number) = EigenDecomposed(eigvals(ed)*c, eigvecs(ed), inveigvecs(ed))
Base.:\(c::Number, ed::EigenDecomposed) = EigenDecomposed(c\eigvals(ed), eigvecs(ed), inveigvecs(ed))
Base.:/(ed::EigenDecomposed, c::Number) = EigenDecomposed(eigvals(ed)/c, eigvecs(ed), inveigvecs(ed))
for T in (AbstractVector, AbstractMatrix)
    @eval function Base.:*(ed::EigenDecomposed, v::$T)
        E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
        P * (Diagonal(E) * (invP * v))
    end
end

function exp_old(ed::EigenDecomposed)
    E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
    expE = exp.(E)
    expA = P * Diagonal(expE) * invP 
    EigenDecomposed(expE, P, invP)
end

LinearAlgebra.lmul!(c::Number, ed::EigenDecomposed) = lmul!(c, eigvals(ed))
LinearAlgebra.rmul!(ed::EigenDecomposed, c::Number) = rmul!(eigvals(ed), c)
LinearAlgebra.ldiv!(c::Number, ed::EigenDecomposed) = ldiv!(c, eigvals(ed))
LinearAlgebra.rdiv!(ed::EigenDecomposed, c::Number) = rdiv!(eigvals(ed), c)

for op in (:exp, :log, :sin, :cos)
    opE = Symbol(op, "E")
    op_eigendecomposed = Symbol(op, "_eigendecomposed")
    op_eigendecomposed! = Symbol(op_eigendecomposed, "!")
    op_eigendecomposed!_doc =
        """
        $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))

        returns the `$op` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`. 
        It does not overwrite `ed` and uses `$opE` and `tmpY` as workspaces.
        """
    @eval begin
        @doc $op_eigendecomposed!_doc
        function $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))
            E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
            @. $opE = $op.(E)
            mul!(tmpY, P, Diagonal($opE))
            mul!(Y, tmpY, invP)
        end
        $op_eigendecomposed(ed::EigenDecomposed) = $op_eigendecomposed!(similar(eigvecs(ed)), ed)
        Base.$op(ed::EigenDecomposed) = $op_eigendecomposed(ed)
    end
end

end

Main.EigenDecomposedMatrices

In [10]:
?EigenDecomposedMatrices.exp_eigendecomposed!

exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))

returns the `exp` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `expE` and `tmpY` as workspaces.


In [11]:
?EigenDecomposedMatrices.log_eigendecomposed!

log_eigendecomposed!(Y, ed::EigenDecomposed, logE=similar(ed.E), tmpY=similar(Y))

returns the `log` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `logE` and `tmpY` as workspaces.


In [12]:
methods(EigenDecomposedMatrices.EigenDecomposed)

# 2 methods for type constructor:
 [1] Main.EigenDecomposedMatrices.EigenDecomposed(E::TE, P::TP, invP::TinvP) where {T, TE<:AbstractVector{T}, TP<:AbstractMatrix{T}, TinvP<:AbstractMatrix{T}}
     @ In[9]:14
 [2] Main.EigenDecomposedMatrices.EigenDecomposed(A::AbstractMatrix)
     @ In[9]:19

In [13]:
methods(EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Matrix{Float64}})

# 1 method for type constructor:
 [1] (var"#ctor-self#"::Type{Main.EigenDecomposedMatrices.EigenDecomposed{T, TE, TP, TinvP}} where {T, TE<:AbstractVector{T}, TP<:AbstractMatrix{T}, TinvP<:AbstractMatrix{T}})(E, P, invP)
     @ In[9]:14

In [14]:
methodswith(EigenDecomposedMatrices.EigenDecomposed)

[1] *(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:35
[2] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:36
[3] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractVector) @ Main.EigenDecomposedMatrices In[9]:40
[4] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractMatrix) @ Main.EigenDecomposedMatrices In[9]:40
[5] /(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:38
[6] \(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:37
[7] convert(::Type{Array}, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:29
[8] cos(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[9] eltype(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:31
[10] exp(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[11] getindex(ed::Main.EigenDecomposedMatrices.EigenDecomposed, I...) @ Main.EigenDecomposedMatrices In[9]:33
[12] log(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[13] parent(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices D:\.julia\packages\Memoization\ut5GT\src\Memoization.jl:162
[14] sin(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[15] size(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:31
[16] eigvals(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:25
[17] eigvecs(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:26
[18] ldiv!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:55
[19] lmul!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:53
[20] rdiv!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:56
[21] rmul!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:54

In [15]:
methods(EigenDecomposedMatrices.exp_eigendecomposed!)

# 3 methods for generic function "exp_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:71
 [2] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE)
     @ In[9]:71
 [3] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE, tmpY)
     @ In[9]:71

In [16]:
methods(EigenDecomposedMatrices.log_eigendecomposed!)

# 3 methods for generic function "log_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:71
 [2] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE)
     @ In[9]:71
 [3] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE, tmpY)
     @ In[9]:71

In [17]:
A = [
    2 -1 0
    -1 2 -1
    0 -1 2
]

edA = EigenDecomposedMatrices.EigenDecomposed(A)

3×3 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  2.0          -1.0  -3.33067e-16
 -1.0           2.0  -1.0
 -3.33067e-16  -1.0   2.0

In [18]:
log(edA)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [19]:
log(A)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [20]:
log(edA) ≈ log(A)

true

In [21]:
n = 2^8
M = 5I + randn(n, n)
v = randn(n)
c = randn()

edM = EigenDecomposedMatrices.EigenDecomposed(M)

Y = similar(eigvecs(edM))
expE = similar(eigvals(edM))
tmpY = similar(Y)

y = similar(eigvals(edM))
alpha = randn()
beta = randn();

In [22]:
edM

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}:
   4.50882-1.99811e-14im  …    -1.93418+1.02755e-14im
  0.178093-1.5888e-13im        0.291791+4.03833e-14im
  0.660934-7.28835e-14im       0.478016+8.06988e-14im
  0.514208-2.46164e-14im       0.420897+1.39345e-13im
  0.315633-1.42527e-13im      -0.991916-3.32215e-15im
 -0.565129-1.30476e-13im  …     0.68231-9.73311e-14im
  0.684523-9.32653e-14im       -1.01822-7.13542e-14im
 -0.245717+1.85644e-14im       -0.15539-1.18862e-13im
 -0.972128-5.32949e-14im      -0.705398-3.22293e-14im
  -1.23254-1.38035e-13im       -0.45892-1.41504e-13im
 -0.338835+2.05102e-15im  …   -0.101063-3.38399e-14im
  -2.63825-5.76997e-14im      -0.300011+7.27317e-15im
  0.267818-8.54684e-14im       0.849471-2.2567e-14im
          ⋮               ⋱            ⋮
 -0.133486+3.82874e-14im       -1.22384-5.27683e-15im
  -1.06467+5.44355e-14im  …     1.07444-1.00114e-13im
   1.73935+1.7277e-14im 

In [23]:
dump(edM)

Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}
  E: Array{ComplexF64}((256,)) ComplexF64[-10.713981718561799 + 0.0im, -10.154389039301256 - 2.424021823386712im, -10.154389039301256 + 2.424021823386712im, -9.723440034384598 - 3.3970170968772115im, -9.723440034384598 + 3.3970170968772115im, -9.032127832245212 + 0.0im, -9.019972086880806 - 2.400029839448304im, -9.019972086880806 + 2.400029839448304im, -8.793253293393999 - 7.761789438318596im, -8.793253293393999 + 7.761789438318596im  …  18.258605642548513 - 7.197681830937633im, 18.258605642548513 + 7.197681830937633im, 18.587800054998098 - 5.614738859029982im, 18.587800054998098 + 5.614738859029982im, 18.83456699611093 - 4.283354547339249im, 18.83456699611093 + 4.283354547339249im, 19.54090238648596 - 2.7461009138097694im, 19.54090238648596 + 2.7461009138097694im, 19.857608966841305 + 0.0im, 22.168239258917858 + 0.0im]
  P: Array{ComplexF64}((256, 256)) ComplexF64[0.056

In [24]:
M ≈ parent(edM) == Matrix(edM)

true

In [25]:
M ≈ edM

true

In [26]:
c*M ≈ c*edM ≈ edM*c

true

In [27]:
c\M ≈ c\edM ≈ edM/c

true

In [28]:
(
    exp(M)
    ≈ exp(edM)
    ≈ EigenDecomposedMatrices.exp_old(edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY)
)

true

In [29]:
@show typeof(y)
(
    exp(M) * v
    ≈ exp(edM) * v
    ≈ EigenDecomposedMatrices.exp_old(edM) * v
    ≈ mul!(y, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM), v)
    ≈ mul!(y, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY), v)
)

typeof(y) = Vector{ComplexF64}


true

In [30]:
@btime edM = EigenDecomposedMatrices.EigenDecomposed(M);

  40.834 ms (26 allocations: 3.53 MiB)


In [31]:
@btime exp($M) * $v
@btime exp($edM) * $v
@btime EigenDecomposedMatrices.exp_old($edM) * $v
@btime mul!($y, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM), $v)
@btime mul!($y, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM, $expE, $tmpY), $v);

  9.925 ms (16 allocations: 3.01 MiB)
  2.641 ms (7 allocations: 2.01 MiB)
  2.609 ms (9 allocations: 2.02 MiB)
  2.466 ms (3 allocations: 1.00 MiB)
  2.324 ms (0 allocations: 0 bytes)


In [32]:
n2 = 2^8
M2 = Symmetric(5I + randn(n2, n2))
v2 = randn(n)
c2 = randn()

edM2 = EigenDecomposedMatrices.EigenDecomposed(M2)

Y2 = similar(eigvecs(edM2))
expE2 = similar(eigvals(edM2))
tmpY2 = similar(Y2)

y2 = similar(eigvals(edM2))
alpha2 = randn()
beta2 = randn();

In [33]:
edM2

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  3.84552     0.735639   -0.888777   …  -1.94169   -1.11377     -0.179077
  0.735639    6.13647     2.36641        0.46021   -1.93826     -0.933242
 -0.888777    2.36641     2.65894       -0.940202  -0.0157323    1.67736
 -1.79558    -1.26612     0.49305        0.449019  -0.421509     1.54008
 -2.04969     0.784308    0.389021      -0.815583  -0.565886     0.370766
  1.73157    -0.385898   -0.513229   …  -0.431498   0.504747    -0.119192
 -1.76546    -0.896505    2.23703       -1.41797    0.669383     2.4564
 -0.136065    0.0741303   0.779564      -0.870817  -1.05158      0.26942
  1.06007    -0.859178   -1.58712        0.818716   1.68292     -2.06501
  0.47115     2.12702    -0.727821       0.311544   0.389826     1.23045
  0.933921    2.8016     -0.146519   …   0.541884   0.341482    -0.569981
  0.503665   -0.989737    0.358613       0.678894  -1.25855  

In [34]:
dump(edM2)

Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}
  E: Array{Float64}((256,)) [-25.571180068391705, -25.10389874230669, -24.71837275242493, -24.14704531547706, -23.549395946381665, -23.27924831353725, -22.86477521997237, -22.34148869945747, -22.06726394345153, -21.69605252089312  …  32.13800956470965, 32.417305346384495, 32.80631583751178, 33.17703854582697, 33.40589623488248, 33.75143436409553, 34.207467542218055, 35.2005032890178, 35.87647737986236, 36.84341690154979]
  P: Array{Float64}((256, 256)) [0.01970303581542531 0.06961126249910145 … 0.10889105047645242 0.06587468385128539; 0.13936923612315275 0.016389422084200564 … 0.08003790520524548 -0.05543473838950966; … ; -0.0813526587874257 -0.08544346354981683 … -0.06647468178125723 -0.06279456372691025; 0.09018581115633151 -0.10369767417719022 … 0.07672144691394629 0.04364369488183982]
  invP: Adjoint{Float64, Matrix{Float64}}
    parent: Array{Float64}((256, 25

In [35]:
M2 ≈ parent(edM2) == Matrix(edM2)

true

In [36]:
M2 ≈ edM2

true

In [37]:
c2*M2 ≈ c2*edM2 ≈ edM2*c2

true

In [38]:
c2\M2 ≈ c2\edM2 ≈ edM2/c2

true

In [39]:
(
    exp(M2)
    ≈ exp(edM2)
    ≈ EigenDecomposedMatrices.exp_old(edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2)
)

true

In [40]:
@show typeof(y2)
(
    exp(M2) * v2
    ≈ exp(edM2) * v2
    ≈ EigenDecomposedMatrices.exp_old(edM2) * v2
    ≈ mul!(y2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2), v2)
    ≈ mul!(y2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2), v2)
)

typeof(y2) = Vector{Float64}


true

In [41]:
@btime edM2 = EigenDecomposedMatrices.EigenDecomposed(M2);

  6.480 ms (14 allocations: 1.59 MiB)


In [42]:
@btime exp($M2) * $v2
@btime exp($edM2) * $v2
@btime EigenDecomposedMatrices.exp_old($edM2) * $v2
@btime mul!($y2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2), $v2)
@btime mul!($y2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2, $expE2, $tmpY2), $v2);

  7.506 ms (19 allocations: 2.60 MiB)
  712.500 μs (6 allocations: 1.00 MiB)
  723.600 μs (8 allocations: 1.01 MiB)
  676.200 μs (3 allocations: 514.17 KiB)
  646.000 μs (0 allocations: 0 bytes)
